 # Lab 5: 품질 평가 - 문화/톤



 존대/격식 수준과 브랜드 보이스 일관성을 평가합니다.



 ## 학습 목표



 - 문화적 적합성 및 톤 일관성 평가 방법 이해

 - 한국어 존대/격식 수준 검증

 - 브랜드 보이스 가이드 기반 평가



 ## 워크샵 전체 흐름



 ```

 Lab 1: 환경 설정 ✅

 ↓

 Lab 2: 번역 기법 비교 ✅

 ↓

 Lab 3: 품질 평가 - 충실도 ✅

 ↓

 Lab 4: 품질 평가 - 용어 일관성 ✅

 ↓

 Lab 5: 품질 평가 - 문화/톤 (현재)

 ↓

 Lab 6: 피드백 기반 재번역

 ↓

 Lab 7: 종합 분석

 ```

 ---

 ## 5.1 환경 설정 및 이전 Lab 결과 로드

In [1]:
import json
import os
from datetime import datetime

import boto3
import pandas as pd


In [2]:
# 설정
REGION = "us-west-2"
OUTPUT_DIR = "lab_outputs"

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=REGION
)

CLAUDE_MODELS = {
    "sonnet": {
        "id": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "name": "Claude Sonnet 4",
    },
    "haiku": {
        "id": "us.anthropic.claude-haiku-4-20250514-v1:0",
        "name": "Claude Haiku 4",
    }
}

DEFAULT_MODEL = "sonnet"
MODEL_ID = CLAUDE_MODELS[DEFAULT_MODEL]["id"]

print(f"Bedrock 클라이언트 초기화 완료")
print(f"사용 모델: {CLAUDE_MODELS[DEFAULT_MODEL]['name']}")


Bedrock 클라이언트 초기화 완료
사용 모델: Claude Sonnet 4


In [3]:
# 유틸리티 함수
def load_results(filename: str) -> dict:
    filepath = os.path.join(OUTPUT_DIR, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"✓ 로드 완료: {filepath}")
    return data

def save_results(data: dict, filename: str) -> str:
    filepath = os.path.join(OUTPUT_DIR, filename)
    data["_metadata"] = {
        "saved_at": datetime.now().isoformat(),
        "model": CLAUDE_MODELS[DEFAULT_MODEL]["name"]
    }
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✓ 저장 완료: {filepath}")
    return filepath

def call_claude(prompt: str, model_id: str = MODEL_ID, max_tokens: int = 2048) -> str:
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = bedrock_client.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body),
        contentType="application/json",
        accept="application/json"
    )
    response_body = json.loads(response["body"].read())
    return response_body["content"][0]["text"]


In [4]:
# Lab 2 결과 로드
zero_shot_data = load_results("translations_zero_shot.json")
few_shot_data = load_results("translations_few_shot.json")
glossary_data = load_results("translations_glossary.json")

# 워크샵 설정 로드 (레퍼런스 포함)
workshop_config = load_results("workshop_config.json")

print(f"\n로드된 번역 결과:")
print(f"  - Zero-shot: {len(zero_shot_data['translations'])}건")
print(f"  - Few-shot: {len(few_shot_data['translations'])}건")
print(f"  - 용어집: {len(glossary_data['translations'])}건")


✓ 로드 완료: lab_outputs/translations_zero_shot.json
✓ 로드 완료: lab_outputs/translations_few_shot.json
✓ 로드 완료: lab_outputs/translations_glossary.json
✓ 로드 완료: lab_outputs/workshop_config.json

로드된 번역 결과:
  - Zero-shot: 6건
  - Few-shot: 6건
  - 용어집: 6건


 ---

 ## 5.2 문화/톤 평가란?



 문화/톤 평가는 번역이 목표 언어의 문화와 브랜드 톤에 얼마나 적합한지 측정합니다.



 ### 평가 항목

 - **자연스러움**: 번역투가 아닌 자연스러운 영어 표현인가?

 - **격식 수준**: 상황에 맞는 격식 수준인가? (formal/informal)

 - **브랜드 보이스**: 삼성의 브랜드 톤과 일치하는가?

 - **지역 적합성**: US/GB 영어 스타일에 맞는가?



 ### 점수 체계 (0-5)



 | 점수 | 판정 | 설명 |

 |------|------|------|

 | 5 | Pass | 톤/문화 완벽 적합 |

 | 4 | Pass | 경미한 톤 조정 필요 |

 | 3 | Review | 격식 불일치 또는 번역투 다수 (검수 필요) |

 | 0-2 | Fail | 부자연스러운 표현, 브랜드 훼손 |

 ---

 ## 5.3 삼성 브랜드 보이스 가이드 정의

In [5]:
# 삼성 브랜드 보이스 가이드 (워크샵용 간소화 버전)
BRAND_VOICE_GUIDE = """
삼성 브랜드 보이스 가이드:

1. 톤: 전문적이면서 친근함
   - 너무 격식적이지 않고, 너무 캐주얼하지도 않음
   - 사용자를 존중하는 정중한 표현

2. 문장 스타일:
   - 명확하고 간결한 문장
   - 능동태 선호
   - 전문 용어는 필요시에만 사용

3. 사용자 호칭:
   - "you/your" 사용 (개인화된 경험 강조)
   - 직접적인 2인칭 표현

4. 피해야 할 표현:
   - 번역투 (직역체)
   - 과도하게 격식적인 표현
   - 모호하거나 장황한 문장
"""

print(BRAND_VOICE_GUIDE)



삼성 브랜드 보이스 가이드:

1. 톤: 전문적이면서 친근함
   - 너무 격식적이지 않고, 너무 캐주얼하지도 않음
   - 사용자를 존중하는 정중한 표현

2. 문장 스타일:
   - 명확하고 간결한 문장
   - 능동태 선호
   - 전문 용어는 필요시에만 사용

3. 사용자 호칭:
   - "you/your" 사용 (개인화된 경험 강조)
   - 직접적인 2인칭 표현

4. 피해야 할 표현:
   - 번역투 (직역체)
   - 과도하게 격식적인 표현
   - 모호하거나 장황한 문장



 ---

 ## 5.4 US vs GB 영어 스타일 차이

In [6]:
# US vs GB 영어 스타일 가이드
US_GB_STYLE_GUIDE = """
US 영어 vs GB 영어 스타일 차이:

| 항목 | US 영어 | GB 영어 |
|------|---------|---------|
| 철자 | synchronized, color | synchronised, colour |
| 용어 | Trash, carrier | Recycle bin, network |
| 톤 | 직접적, 간결 | 좀 더 형식적 |
| 소유격 | a Samsung account | my Samsung account |
| 동사 | remove | delete |
"""

print(US_GB_STYLE_GUIDE)



US 영어 vs GB 영어 스타일 차이:

| 항목 | US 영어 | GB 영어 |
|------|---------|---------|
| 철자 | synchronized, color | synchronised, colour |
| 용어 | Trash, carrier | Recycle bin, network |
| 톤 | 직접적, 간결 | 좀 더 형식적 |
| 소유격 | a Samsung account | my Samsung account |
| 동사 | remove | delete |



In [7]:
# 레퍼런스 데이터에서 US/GB 차이 확인
print("=" * 80)
print("실제 레퍼런스에서 US/GB 차이 확인")
print("=" * 80)

for item in workshop_config["faq_items"]:
    if item["en_us"] != item["en_gb"]:
        print(f"\n[{item['id']}]")
        print(f"  한국어: {item['ko']}")
        print(f"  US: {item['en_us']}")
        print(f"  GB: {item['en_gb']}")


실제 레퍼런스에서 US/GB 차이 확인

[IDS_FAQ_SC_MAIN_HEADER_02]
  한국어: 삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?
  US: Do I need a Samsung account to use Samsung Cloud?
  GB: Do I need my Samsung account to use Samsung Cloud?

[IDS_FAQ_SC_MAIN_HEADER_03]
  한국어: 삼성 계정을 단말에서 제거하면 어떻게 되나요?
  US: What happens if I remove my Samsung account?
  GB: What happens if I delete my Samsung account?

[IDS_FAQ_SC_ACCESSING_TEXT]
  한국어: 삼성 클라우드 웹페이지 접속 후 Contact us를 통해 '1대1 문의하기'로 문의해주시면 사용자 정보 확인 후 차단 해제가 가능합니다.
  US: After accessing the Samsung Cloud web page, contact us through the "1:1 inquiry" in "Contact us". We will unlock your Samsung Cloud after confirming your user information.
  GB: After accessing the Samsung Cloud web page, contact us through the "1:1 enquiry" in "Contact us". We will unlock your Samsung Cloud after confirming your user information.

[IDS_FAQ_SC_MAIN_HEADER_13]
  한국어: 갤럭시 단말이 현재 없습니다. 삼성 서비스에 저장 및 동기화했던 데이터나 정보를 확인할 수 있는 방법이 있나요?
  US: I do not have a Galaxy device now. Is there a way to check my

 ---

 ## 5.5 문화/톤 평가 프롬프트 설계

In [8]:
CULTURE_TONE_EVAL_PROMPT = """당신은 번역 품질 평가 전문가입니다.
아래 원문, 번역문, 브랜드 가이드를 참고하여 문화/톤 적합성을 평가해주세요.

## 원문 (한국어)
{source}

## 번역문 (영어)
{translation}

## 레퍼런스 번역 (US)
{reference_us}

## 레퍼런스 번역 (GB)
{reference_gb}

## 브랜드 보이스 가이드
{brand_guide}

## 평가 기준
1. 자연스러움: 번역투가 아닌 자연스러운 영어 표현인가?
2. 격식 수준: 브랜드에 맞는 전문적이면서 친근한 톤인가?
3. 브랜드 보이스: 삼성의 브랜드 톤과 일치하는가?
4. 레퍼런스 유사도: US 또는 GB 레퍼런스와 얼마나 유사한가?

## 점수 기준
- 5점: 톤/문화 완벽 적합, 레퍼런스와 거의 동일
- 4점: 경미한 톤 조정 필요
- 3점: 격식 불일치 또는 번역투 (검수 필요)
- 0-2점: 부자연스러운 표현, 브랜드 훼손

## 출력 형식
반드시 아래 JSON 형식으로만 응답하세요:
```json
{{
  "score": <0-5 사이 정수>,
  "verdict": "<Pass|Review|Fail>",
  "naturalness": "<자연스러움|약간 어색|번역투>",
  "formality": "<적절|너무 격식적|너무 캐주얼>",
  "brand_voice_match": <true|false>,
  "closer_to": "<US|GB|Neither>",
  "issues": ["발견된 문제점"],
  "suggestion": "개선 제안",
  "reasoning": "평가 근거를 한국어로 설명"
}}
```"""


 ---

 ## 5.6 평가 함수 정의

In [9]:
def evaluate_culture_tone(source: str, translation: str, reference_us: str, reference_gb: str) -> dict:
    """문화/톤 평가를 수행하고 결과를 반환합니다."""
    
    prompt = CULTURE_TONE_EVAL_PROMPT.format(
        source=source,
        translation=translation,
        reference_us=reference_us,
        reference_gb=reference_gb,
        brand_guide=BRAND_VOICE_GUIDE
    )
    
    response = call_claude(prompt)
    
    try:
        if "```json" in response:
            json_str = response.split("```json")[1].split("```")[0].strip()
        elif "```" in response:
            json_str = response.split("```")[1].split("```")[0].strip()
        else:
            json_str = response.strip()
        result = json.loads(json_str)
    except (json.JSONDecodeError, IndexError) as e:
        result = {
            "score": 0, 
            "verdict": "Fail", 
            "naturalness": "평가 실패",
            "formality": "평가 실패",
            "brand_voice_match": False,
            "closer_to": "Neither",
            "issues": ["파싱 실패"], 
            "suggestion": "", 
            "reasoning": str(e)
        }
    
    return result


In [10]:
# 평가 테스트
test_item = workshop_config["faq_items"][0]
test_translation = zero_shot_data["translations"][0]["translation"]

print("평가 테스트")
print(f"원문: {test_item['ko']}")
print(f"번역: {test_translation}")
print(f"레퍼런스(US): {test_item['en_us']}")
print(f"레퍼런스(GB): {test_item['en_gb']}")
print("-" * 50)

test_result = evaluate_culture_tone(
    test_item['ko'], 
    test_translation, 
    test_item['en_us'], 
    test_item['en_gb']
)
print(f"점수: {test_result['score']}")
print(f"자연스러움: {test_result['naturalness']}")
print(f"레퍼런스 유사: {test_result['closer_to']}")
print(f"근거: {test_result['reasoning']}")


평가 테스트
원문: 삼성 클라우드를 사용하려면 삼성 계정이 필요한가요?
번역: Do I need a Samsung account to use Samsung Cloud?
레퍼런스(US): Do I need a Samsung account to use Samsung Cloud?
레퍼런스(GB): Do I need my Samsung account to use Samsung Cloud?
--------------------------------------------------
점수: 5
자연스러움: 자연스러움
레퍼런스 유사: US
근거: 번역문은 US 레퍼런스와 완전히 동일하며, 삼성의 브랜드 보이스 가이드에 완벽하게 부합합니다. 'Do I need a Samsung account~' 표현은 자연스러운 영어이고, 전문적이면서도 친근한 톤을 유지하고 있습니다. 직접적인 2인칭 표현('I')을 사용하여 개인화된 경험을 강조하고 있으며, 명확하고 간결한 문장 구조로 되어 있습니다. 번역투나 과도한 격식체 없이 자연스러운 영어 표현을 사용했습니다.


 ---

 ## 5.7 세 가지 번역 기법 평가 실행

In [11]:
def run_culture_tone_evaluation(translations: list, faq_items: list, method_name: str) -> list:
    """번역 목록에 대해 문화/톤 평가를 실행합니다."""
    
    print(f"\n{method_name} 문화/톤 평가 중...")
    print("=" * 80)
    
    results = []
    
    # faq_items를 id로 매핑
    faq_map = {item["id"]: item for item in faq_items}
    
    for item in translations:
        faq_item = faq_map.get(item["id"], {})
        reference_us = faq_item.get("en_us", "")
        reference_gb = faq_item.get("en_gb", "")
        
        eval_result = evaluate_culture_tone(
            item["source"], 
            item["translation"],
            reference_us,
            reference_gb
        )
        
        result = {
            "id": item["id"],
            "source": item["source"],
            "translation": item["translation"],
            "reference_us": reference_us,
            "reference_gb": reference_gb,
            "method": item["method"],
            "score": eval_result["score"],
            "verdict": eval_result["verdict"],
            "naturalness": eval_result.get("naturalness", ""),
            "formality": eval_result.get("formality", ""),
            "brand_voice_match": eval_result.get("brand_voice_match", False),
            "closer_to": eval_result.get("closer_to", "Neither"),
            "issues": eval_result["issues"],
            "suggestion": eval_result["suggestion"],
            "reasoning": eval_result["reasoning"],
            "eval_type": "culture_tone",
            "timestamp": datetime.now().isoformat()
        }
        results.append(result)
        
        status = "✓" if eval_result["verdict"] == "Pass" else "△" if eval_result["verdict"] == "Review" else "✗"
        print(f"\n[{item['id']}] {status} 점수: {eval_result['score']}/5")
        print(f"  번역: {item['translation'][:50]}...")
        print(f"  자연스러움: {eval_result.get('naturalness', 'N/A')}")
        print(f"  레퍼런스 유사: {eval_result.get('closer_to', 'N/A')}")
    
    return results


In [12]:
# Zero-shot 평가
faq_items = workshop_config["faq_items"]

zero_shot_eval = run_culture_tone_evaluation(
    zero_shot_data["translations"], 
    faq_items,
    "Zero-shot"
)



Zero-shot 문화/톤 평가 중...

[IDS_FAQ_SC_MAIN_HEADER_02] ✓ 점수: 5/5
  번역: Do I need a Samsung account to use Samsung Cloud?...
  자연스러움: 자연스러움
  레퍼런스 유사: US

[IDS_FAQ_SC_MAIN_HEADER_03] △ 점수: 3/5
  번역: What happens when you remove a Samsung account fro...
  자연스러움: 약간 어색
  레퍼런스 유사: Neither

[IDS_FAQ_SC_ACCESSING_TEXT] △ 점수: 3/5
  번역: After accessing the Samsung Cloud webpage, please ...
  자연스러움: 약간 어색
  레퍼런스 유사: Neither

[IDS_FAQ_SC_MAIN_HEADER_13] ✓ 점수: 4/5
  번역: I don't currently have a Galaxy device. Is there a...
  자연스러움: 자연스러움
  레퍼런스 유사: US

[IDS_FAQ_GO_DEVICE_NOTES_UL_01] ✓ 점수: 4/5
  번역: Support availability may vary depending on the spe...
  자연스러움: 자연스러움
  레퍼런스 유사: Neither

[IDS_FAQ_GO_MAIN_HEADER_27] △ 점수: 3/5
  번역: Where can I go to check the cloud recycle bin?...
  자연스러움: 약간 어색
  레퍼런스 유사: GB


In [13]:
# Few-shot 평가
few_shot_eval = run_culture_tone_evaluation(
    few_shot_data["translations"], 
    faq_items,
    "Few-shot"
)



Few-shot 문화/톤 평가 중...

[IDS_FAQ_SC_MAIN_HEADER_02] ✓ 점수: 5/5
  번역: Do I need a Samsung account to use Samsung Cloud?...
  자연스러움: 자연스러움
  레퍼런스 유사: US

[IDS_FAQ_SC_MAIN_HEADER_03] △ 점수: 3/5
  번역: What happens if I remove my Samsung account from m...
  자연스러움: 약간 어색
  레퍼런스 유사: Neither

[IDS_FAQ_SC_ACCESSING_TEXT] △ 점수: 3/5
  번역: After accessing the Samsung Cloud webpage, you can...
  자연스러움: 약간 어색
  레퍼런스 유사: Neither

[IDS_FAQ_SC_MAIN_HEADER_13] ✓ 점수: 4/5
  번역: I don't currently have a Galaxy device. Is there a...
  자연스러움: 자연스러움
  레퍼런스 유사: US

[IDS_FAQ_GO_DEVICE_NOTES_UL_01] ✓ 점수: 4/5
  번역: Support may vary depending on the specific device,...
  자연스러움: 자연스러움
  레퍼런스 유사: Neither

[IDS_FAQ_GO_MAIN_HEADER_27] △ 점수: 3/5
  번역: Where can I go to check my Cloud Trash?...
  자연스러움: 약간 어색
  레퍼런스 유사: Neither


In [14]:
# 용어집 적용 평가
glossary_eval = run_culture_tone_evaluation(
    glossary_data["translations"], 
    faq_items,
    "용어집 적용"
)



용어집 적용 문화/톤 평가 중...

[IDS_FAQ_SC_MAIN_HEADER_02] ✓ 점수: 5/5
  번역: Do I need a Samsung account to use Samsung Cloud?...
  자연스러움: 자연스러움
  레퍼런스 유사: US

[IDS_FAQ_SC_MAIN_HEADER_03] △ 점수: 3/5
  번역: What happens when you remove your Samsung account ...
  자연스러움: 약간 어색
  레퍼런스 유사: Neither

[IDS_FAQ_SC_ACCESSING_TEXT] △ 점수: 3/5
  번역: After accessing the Samsung Cloud webpage, please ...
  자연스러움: 약간 어색
  레퍼런스 유사: Neither

[IDS_FAQ_SC_MAIN_HEADER_13] ✓ 점수: 4/5
  번역: I don't currently have a Galaxy device. Is there a...
  자연스러움: 자연스러움
  레퍼런스 유사: US

[IDS_FAQ_GO_DEVICE_NOTES_UL_01] ✓ 점수: 4/5
  번역: Support may vary depending on the specific device,...
  자연스러움: 자연스러움
  레퍼런스 유사: Neither

[IDS_FAQ_GO_MAIN_HEADER_27] △ 점수: 3/5
  번역: Where can I go to check the cloud Trash?...
  자연스러움: 약간 어색
  레퍼런스 유사: Neither


 ---

 ## 5.8 레퍼런스 유사도 분석



 각 번역이 US와 GB 레퍼런스 중 어느 쪽에 더 가까운지 분석합니다.

In [15]:
print("=" * 80)
print("레퍼런스 유사도 분석")
print("=" * 80)

def count_closer_to(eval_results: list) -> dict:
    counts = {"US": 0, "GB": 0, "Neither": 0}
    for r in eval_results:
        closer = r.get("closer_to", "Neither")
        if closer in counts:
            counts[closer] += 1
    return counts

zero_counts = count_closer_to(zero_shot_eval)
few_counts = count_closer_to(few_shot_eval)
glossary_counts = count_closer_to(glossary_eval)

print(f"\n{'기법':<15} {'US':<8} {'GB':<8} {'Neither':<8}")
print("-" * 40)
print(f"{'Zero-shot':<15} {zero_counts['US']:<8} {zero_counts['GB']:<8} {zero_counts['Neither']:<8}")
print(f"{'Few-shot':<15} {few_counts['US']:<8} {few_counts['GB']:<8} {few_counts['Neither']:<8}")
print(f"{'용어집':<15} {glossary_counts['US']:<8} {glossary_counts['GB']:<8} {glossary_counts['Neither']:<8}")

print("\n" + "-" * 40)
print("해석: 현재 번역은 특정 지역(US/GB)을 타겟팅하지 않아 일관성이 부족함")
print("→ Lab 6에서 US/GB 타겟팅 재번역으로 개선 예정")


레퍼런스 유사도 분석

기법              US       GB       Neither 
----------------------------------------
Zero-shot       2        1        3       
Few-shot        2        0        4       
용어집             2        0        4       

----------------------------------------
해석: 현재 번역은 특정 지역(US/GB)을 타겟팅하지 않아 일관성이 부족함
→ Lab 6에서 US/GB 타겟팅 재번역으로 개선 예정


 ---

 ## 5.9 평가 결과 비교

In [16]:
def summarize_scores(eval_results: list, method_name: str) -> dict:
    scores = [r["score"] for r in eval_results]
    verdicts = [r["verdict"] for r in eval_results]
    brand_matches = [r.get("brand_voice_match", False) for r in eval_results]
    
    return {
        "method": method_name,
        "avg_score": sum(scores) / len(scores),
        "pass_count": verdicts.count("Pass"),
        "review_count": verdicts.count("Review"),
        "fail_count": verdicts.count("Fail"),
        "brand_match_count": sum(brand_matches),
        "total": len(scores)
    }

zero_summary = summarize_scores(zero_shot_eval, "Zero-shot")
few_summary = summarize_scores(few_shot_eval, "Few-shot")
glossary_summary = summarize_scores(glossary_eval, "용어집")


In [17]:
print("=" * 80)
print("문화/톤 평가 결과 비교")
print("=" * 80)

print(f"\n{'기법':<15} {'평균점수':<10} {'Pass':<8} {'Review':<8} {'Fail':<8} {'브랜드일치':<10}")
print("-" * 65)

for summary in [zero_summary, few_summary, glossary_summary]:
    print(f"{summary['method']:<15} {summary['avg_score']:<10.2f} {summary['pass_count']:<8} {summary['review_count']:<8} {summary['fail_count']:<8} {summary['brand_match_count']:<10}")

print("=" * 80)


문화/톤 평가 결과 비교

기법              평균점수       Pass     Review   Fail     브랜드일치     
-----------------------------------------------------------------
Zero-shot       3.67       3        3        0        3         
Few-shot        3.67       3        3        0        3         
용어집             3.67       3        3        0        3         


 ---

 ## 5.10 세 가지 평가 종합 비교



 Lab 3(충실도), Lab 4(용어), Lab 5(톤) 결과를 종합 비교합니다.

In [18]:
# 이전 Lab 결과 로드
faithfulness_data = load_results("eval_faithfulness.json")
terminology_data = load_results("eval_terminology.json")

print("=" * 80)
print("세 가지 평가 축 종합 비교")
print("=" * 80)

print(f"\n{'기법':<15} {'충실도':<10} {'용어':<10} {'톤':<10} {'평균':<10}")
print("-" * 55)

for key, name in [("zero_shot", "Zero-shot"), ("few_shot", "Few-shot"), ("glossary", "용어집")]:
    faith = faithfulness_data["summary"][key]["avg_score"]
    term = terminology_data["summary"][key]["avg_score"]
    
    if key == "zero_shot":
        tone = zero_summary["avg_score"]
    elif key == "few_shot":
        tone = few_summary["avg_score"]
    else:
        tone = glossary_summary["avg_score"]
    
    avg = (faith + term + tone) / 3
    
    print(f"{name:<15} {faith:<10.2f} {term:<10.2f} {tone:<10.2f} {avg:<10.2f}")

print("\n" + "-" * 55)
print("해석:")
print("  - 충실도: Zero-shot이 높음 (원문에 충실)")
print("  - 용어: 용어집 적용이 높음 (일관된 용어)")
print("  - 톤: Few-shot/용어집이 높음 (스타일 학습)")
print("=" * 80)


✓ 로드 완료: lab_outputs/eval_faithfulness.json
✓ 로드 완료: lab_outputs/eval_terminology.json
세 가지 평가 축 종합 비교

기법              충실도        용어         톤          평균        
-------------------------------------------------------
Zero-shot       4.83       4.17       3.67       4.22      
Few-shot        4.50       4.83       3.67       4.33      
용어집             4.50       5.00       3.67       4.39      

-------------------------------------------------------
해석:
  - 충실도: Zero-shot이 높음 (원문에 충실)
  - 용어: 용어집 적용이 높음 (일관된 용어)
  - 톤: Few-shot/용어집이 높음 (스타일 학습)


 ---

 ## 5.11 평가 결과 저장

In [19]:
culture_tone_results = {
    "zero_shot": zero_shot_eval,
    "few_shot": few_shot_eval,
    "glossary": glossary_eval,
    "summary": {
        "zero_shot": zero_summary,
        "few_shot": few_summary,
        "glossary": glossary_summary
    }
}

save_results(culture_tone_results, "eval_culture_tone.json")


✓ 저장 완료: lab_outputs/eval_culture_tone.json


'lab_outputs/eval_culture_tone.json'

 ---

 ## 5.12 Lab 5 완료

In [20]:
print("=" * 80)
print("Lab 5: 품질 평가 - 문화/톤 완료")
print("=" * 80)

print("\n[평가 결과 요약]")
for summary in [zero_summary, few_summary, glossary_summary]:
    print(f"  - {summary['method']}: 평균 {summary['avg_score']:.2f}점, 브랜드 일치 {summary['brand_match_count']}건")

print("\n[핵심 인사이트]")
print("  - 현재 번역은 US/GB 지역을 타겟팅하지 않아 일관성 부족")
print("  - Few-shot과 용어집이 브랜드 톤 일치도가 높음")
print("  - 레퍼런스와 완전히 일치하려면 지역 타겟팅 필요")

print("\n[저장된 파일]")
print(f"  - {OUTPUT_DIR}/eval_culture_tone.json")

print("\n[다음 단계]")
print("  → Lab 6: 피드백 기반 재번역")
print("    평가 결과 + US/GB 타겟팅으로 레퍼런스에 가까운 번역 생성")

print("\n" + "=" * 80)


Lab 5: 품질 평가 - 문화/톤 완료

[평가 결과 요약]
  - Zero-shot: 평균 3.67점, 브랜드 일치 3건
  - Few-shot: 평균 3.67점, 브랜드 일치 3건
  - 용어집: 평균 3.67점, 브랜드 일치 3건

[핵심 인사이트]
  - 현재 번역은 US/GB 지역을 타겟팅하지 않아 일관성 부족
  - Few-shot과 용어집이 브랜드 톤 일치도가 높음
  - 레퍼런스와 완전히 일치하려면 지역 타겟팅 필요

[저장된 파일]
  - lab_outputs/eval_culture_tone.json

[다음 단계]
  → Lab 6: 피드백 기반 재번역
    평가 결과 + US/GB 타겟팅으로 레퍼런스에 가까운 번역 생성

